<a href="https://colab.research.google.com/github/mateus-miguel/case-dadosfera/blob/main/ETL%201%20-%20Limpeza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [ ]:
# --- Formato JSONL raw inicial ---
# {'docid': int, 'title': str, 'text': str}
# { ... }
# { ... }

# Ou seja, não é válido como JSON ao todo [{}, {}, ...] nem tem vírgulas, só é formado por vários JSON empilhados a cada linha.

In [ ]:
import os
import boto3
import json

# Chaves de acesso AWS


# Locais do Bucket e File dentro da AWS S3
dados = []
bucket = 'dadosfera-datalake'
file_jsonl = 'bronze/products_raw.jsonl'

s3 = boto3.client('s3')

obj = s3.get_object(
    Bucket=bucket,
    Key=file_jsonl
)

dados = []

for line in obj["Body"].iter_lines():
  if line: # pula linhas vazias
    dados.append(json.loads(line))

# Não funciona pois é JSONL não JSON
# data = json.loads(obj["Body"].read())

len_total = len(dados)
print(dados[0])
print(f'Total de linhas do JSONL: {len_total}')

{'docid': 1, 'title': 'FYY Leather Case with Mirror for Samsung Galaxy S8 Plus, Leather Wallet Flip Folio Case with Mirror and Wrist Strap for Samsung Galaxy S8 Plus Black', 'text': 'Product Description Premium PU Leather Top quality. Made with Premium PU Leather. Receiver design. Accurate cut-out for receiver. Convenient to Answer the phone without open the case. Hand strap makes it easy to carry around. RFID Technique RFID Technique: Radio Frequency Identification technology, through radio signals to identify specific targets and to read and copy electronic data. Most Credit Cards, Debit Cards, ID Cards are set-in the RFID chip, the RFID reader can easily read the cards information within 10 feet(about 3m) without touching them. This case is designed to protect your cards information from stealing with blocking material of RFID shielding technology. 100% Handmade 100% Handmade. Perfect craftmanship and reinforced stitching makes it even more durable. Sleek, practical and elegant with

In [ ]:
# --- LIMPEZA DE DADOS FALTANTES ---
# Alguns produtos não têm title ou text (empty string), ou ambos, nesse caso são filtrados (removidos)
# P.ex. docid: 5, 10, 13, 21, 39, 63, 77, 87, 99 não têm 'text', já docid: 65, 66, 97 não têm ambos 'title' e 'text'

dados = [item for item in dados if item['title'] != '' and item['text'] != '']

print(f'Total de produtos após filtro de nulos: {len(dados)}')
print(f'Porcentagem de produtos nulos: {(1 - len(dados) / len_total) * 100 :.2f} %')

# Normalizando outros objetos como aspas tipográficas “, ”, ‘, ’ pelas aspas comuns e –, • por string vazia
def normalize(text: str) -> str:
  replacements = {
      "“": '"', "”": '"', "‘": "'", "’": "'", "´": "'", "｀": "'",
      "–": " ", "—": " ", "•": " ", "*": " ", "➤": " ", "✔": " ", "™": " ", "®": " ",
      "Previous page": " ", "Next page": " ", "Product Description": " ", "Read more": " ", "From the manufacturer": " "
  }

  for old, new in replacements.items():
    text = text.replace(old, new)

  return text

changed = 0
unchanged = 0

for item in dados:
  text = item.get('text')

  normalized = normalize(item['text'])

  if normalized != text:
    changed += 1
    item['text'] = normalized
  else:
    unchanged += 1

print(f"Descrições alteradas: {changed}")
print(f"Descrições não alteradas: {unchanged}")
# 286214 descrições alteradas com essa normalização, cerca de 31.96%

# Filtrando para apenas 100k produtos iniciais (mínimo do case)
linhas = 100000
dados_reduced = dados[0:linhas]

# Colocando o novo arquivo limpo no bucket silver do bucket S3
s3.put_object(
    Bucket=bucket,
    Key=f'silver/products_clean_{linhas}rows.json',
    Body=json.dumps(dados_reduced, ensure_ascii=False).encode('utf-8')
)

Total de produtos após filtro de nulos: 895673
Porcentagem de produtos nulos: 19.93 %
Descrições alteradas: 0
Descrições não alteradas: 895673


{'ResponseMetadata': {'RequestId': 'C7HAJQTTP06NGFHY',
  'HostId': 'IRvSmblV1Ddscg+Kb+EkcndQlaI8ete1FUc1fOo7bG3CHUT4sEjdDVT/Rpn70gCq+LQgU6WEGZU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'IRvSmblV1Ddscg+Kb+EkcndQlaI8ete1FUc1fOo7bG3CHUT4sEjdDVT/Rpn70gCq+LQgU6WEGZU=',
   'x-amz-request-id': 'C7HAJQTTP06NGFHY',
   'date': 'Fri, 16 Jan 2026 15:53:19 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5f2918346e3c0c5e38a31fba58b30033"',
   'x-amz-checksum-crc32': 'FRMiRw==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"5f2918346e3c0c5e38a31fba58b30033"',
 'ChecksumCRC32': 'FRMiRw==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [ ]:
# --- BLOCO DE TESTE PARA ARQUIVO JSONL LOCAL ---

arquivo = './products_raw.jsonl'

# Contando total de linhas/produtos dentro do arquivo
with open(arquivo, 'r', encoding='utf-8') as f:
  total_linhas = sum(1 for _ in f)

print(total_linhas)

# Convertendo o arquivo JSONL ou TXT em um arquivo com todos os JSON dos produtos
with open(arquivo, 'r', encoding='utf-8') as f:
  for i, linha in enumerate(f, start=1):
    linha = linha.strip() # tira \n de nova linha
    if not linha: # se não for vazia
      continue

    try:
      obj = json.loads(linha)
      dados.append(obj)

    # Alguma ou algums linhas do JSONL geram um erro JSONDecodeError talvez por falta de fechar aspas "", então temos que tratar o erro try/except
    except json.JSONDecodeError as e:
      print(f'Erro na linha {i}: {e}')
      print(linha[:300])
      break

# Limpeza de dados pois algumas linhas têm title ou text vazios (ou ambos)
dados = [doc for doc in dados if doc['title'] != '' and doc['text'] != ''] # filtrado
print(len(dados)) # total de produtos após filtro de nulos

novos_dados = dados[0:100000]

# Gerar um único JSON de saída com todos os produtos cadastrados
with open('products_jsonl_1000.json', 'w', encoding='utf-8') as f:
  json.dump(novos_dados, f, ensure_ascii=False, indent=2)